In [1]:
import pickle
import os
import argparse
import torch
from jax import random
import json
import datetime
from src.losses import sse_loss
from src.helper import calculate_exact_ggn, tree_random_normal_like
from src.sampling.predictive_samplers import sample_predictive, sample_hessian_predictive
from jax import numpy as jnp
import jax
from jax import flatten_util
import matplotlib.pyplot as plt
import tree_math as tm
from src.laplace.diagonal import hutchinson_diagonal

In [2]:
def f(x):
    return jnp.sin(5 * x + 1) #+ jnp.cos(25 * x + 1) + jnp.exp(0.1 * x) + 5
param_dict = pickle.load(open("../checkpoints/syntetic_regression.pickle", "rb"))
params = param_dict['params']
alpha = param_dict['alpha']
rho = param_dict['rho']
x_train, y_train, x_val, y_val, model, D = param_dict["train_stats"]['x_train'],param_dict["train_stats"]['y_train'],param_dict["train_stats"]['x_val'],param_dict["train_stats"]['y_val'],param_dict["train_stats"]['model'], param_dict["train_stats"]['n_params']

In [3]:
sample_key = jax.random.PRNGKey(100)
model_fn = lambda params, x: model.apply(params, x[None, ...])[0]
n_params = D 
def sse_loss(preds, y):
    residual = preds - y
    return 0.5 * jnp.sum(residual**2)

ggn = calculate_exact_ggn(sse_loss, model_fn, params, x_train, y_train, n_params)
true_diag = jnp.diag(ggn)

In [4]:
model_fn = model.apply
gvp_batch_size = 25
N = x_train.shape[0]//gvp_batch_size
data_array = x_train[: N * gvp_batch_size].reshape((N, gvp_batch_size)+ x_train.shape[1:])
n_samples = 10000
diag_hutch = hutchinson_diagonal(model_fn, params, gvp_batch_size, n_samples, sample_key, data_array, "regression", num_levels=5, computation_type="serial")
diag_hutch, _ = jax.flatten_util.ravel_pytree(diag_hutch)
print(jnp.linalg.norm(diag_hutch - true_diag) / jnp.linalg.norm(true_diag))

0.013861573


In [5]:
from src.laplace.diagonal import exact_diagonal
output_dim = 1
exact_diag = exact_diagonal(model.apply, params, output_dim, x_train, x_train.shape[0], "regression")

In [6]:
exact_diag, _ = jax.flatten_util.ravel_pytree(exact_diag)
print(jnp.linalg.norm(exact_diag - true_diag) / jnp.linalg.norm(true_diag))

6.138341e-08
